In [50]:
import pandas as pd
import numpy as np
import math

In [51]:
articles_df = pd.read_csv('data/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']

In [52]:
interactions_df = pd.read_csv('data/users_interactions.csv')

In [53]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [54]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [55]:
interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

In [56]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

In [57]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [58]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full_df = interactions_full_df.reset_index()

In [59]:
from sklearn.model_selection import train_test_split

split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()


Задание 6.1

Найдите оценку взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372. Результат округлите до двух знаков после точки-разделителя.

In [60]:
interactions_train_df[(interactions_train_df.personId == '-1032019229384696495') & (interactions_train_df.contentId == '943818026930898372')]

,personId,contentId,eventStrength,last_timestamp
653,-1032019229384696495,943818026930898372,2.321928,1464092042


Теперь давайте попробуем применить memory-based-подход коллаборативной фильтрации.

Примечание. Данных достаточно много, поэтому для увеличения скорости работы преобразуйте таблицу в массив numpy.

Задание 6.2

Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.

In [65]:
ratings = pd.pivot_table( interactions_train_df, values="eventStrength", index="personId", columns="contentId", ).fillna(0)
ratings_m = ratings.values
ratings.values.mean()

0.016668620737604063

Перейдём к реализации коллаборативной фильтрации. Ранее мы делали это с помощью библиотеки surprise, однако это не всегда удобно, так как эта библиотека имеет ограниченное количество метрик для оценки качества и небольшой потенциал для более тонкой настройки алгоритма. Поэтому давайте попробуем реализовать алгоритмы коллаборативной фильтрации «с нуля». Такая практика применяется, если необходимо выстроить более сложную систему, чем могут предложить готовые модули. Кроме того, «ручная» реализация алгоритмов позволит лучше понять принцип их работы.

Задание 6.3

Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, полученной в предыдущем задании. Для каждой пары учитывайте только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас). Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам.

In [66]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))
for i in (range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
     
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]

        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]

similarity_users[3,40]

/Users/egor/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/egor/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/Users/egor/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/egor/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Users/egor/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2705: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Users/egor/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/egor/anaconda3/

-0.3333333333333333

Теперь у нас есть матрицы схожести пользователей. Их можно использовать для построения рекомендаций. Чтобы это сделать, надо реализовать следующий алгоритм.

Для каждого пользователя:

Найти пользователей с похожестью больше 0.
Для каждой статьи вычислить долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
Порекомендовать статьи (не более 10) с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).


Задание 6.4

Постройте рекомендательную систему по алгоритму, описанному выше. Найдите первую рекомендацию для строки 35 (если считать с нуля).

In [72]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)
 
interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)
 
interactions['true_test'] = [ [] if x is np.NaN else x for x in interactions['true_test'] ]

prediction_user_based = []
for i in range(len(similarity_users)):
    users_sim = similarity_users[i] > 0
    if not any(users_sim):
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i]["true_train"])][:10]
        prediction_user_based.append(list(recommend))
interactions['prediction_user_based'] = prediction_user_based
prediction_user_based[35][0]

'-5148591903395022444'

После того как сделаны предсказания, можно вычислить качество по метрике, которую мы определили в предыдущем модуле при решении этой задачи:

In [77]:
def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()
    
calc_precision('prediction_user_based')


0.0045585587099675725

Задание 6.6

Теперь реализуем рекомендательную систему с использованием SVD.

Разложите матрицу взаимодействий пользователей со статьями с помощью функции svd из модуля scipy. Найдите максимальное значение в получившейся матрице U. Результат округлите до двух знаков после точки-разделителя.

In [98]:
from scipy.linalg import svd

U, s, Vh = svd(ratings)
U.max()

0.7071067811865471

Значения матрицы с сингулярными числами отсортированы по убыванию. Допустим, мы хотим оставить только первые 100 компонент и получить скрытые представления размерности 100. Для этого необходимо оставить 100 столбцов в матрице U, только первые 100 значений из sigma (и сделать из них диагональную матрицу) и 100 столбцов в матрице V. Затем необходимо перемножить преобразованные матрицы.

Задание 6.7

Найдите сумму всех элементов в новой сингулярной матрице. Ответ округлите до двух знаков после точки-разделителя.

In [107]:
U_s = U[:, :100]
s_s = np.diag(s[:100,])
Vh_s = Vh[:100, :]
mx = U_s @ s_s @ Vh_s



In [112]:
s_s.sum()

2096.432772165699

Теперь мы можем сделать предсказание по полученной матрице.

Примечание. Помните, что не нужно учитывать статьи, которые уже были просмотрены пользователем.

Найдите для каждого пользователя статьи с наибольшими оценками в восстановленной матрице.

Задание 6.8

1 point possible (graded)
Вычислите качество полученного предсказания, используя всё ту же метрику точности. Ответ округлите до трёх знаков после точки-разделителя.

In [126]:
new_ratings = pd.DataFrame(
    mx, 
    index=ratings.index, 
    columns=ratings.columns
    )

top_k = 10
predictions = []
for personId in interactions.index:
    prediction = (
        new_ratings.loc[personId].sort_values(ascending=False).index.values
    )

    predictions.append(
        list(
            prediction[
                ~np.in1d(prediction, interactions.loc[personId, "true_train"])
            ]
        )[:top_k]
    )

interactions["prediction_svd"] = predictions

calc_precision("prediction_svd")

0.012023152289763838

Итак, мы реализовали два алгоритма коллаборативной фильтрации буквально с нуля! Теперь для полноты картины давайте реализуем на этих данных гибридную модель и посмотрим, какое качество получится. Для этого воспользуемся уже изученной библиотекой LightFM.

Задание 6.9

Возьмите матрицу, подготовленную в задании 6.1. Преобразуйте её в разреженную матрицу:

In [129]:
from scipy.sparse import csr_matrix
ratings_matrix = csr_matrix(ratings)

Воспользовавшись функцией random_train_test_split() из библиотеки lightfm, разделите данные на валидационную и обучающую выборки в соотношении 1:2 (30% на валидационную выборку, 70% на обучающую). В качестве значения параметра random_state возьмите число 13.

Обучите модель LightFM со 100 компонентами, параметром random_state = 13, темпом обучения 0.05 и функцией потерь 'warp'. Обратите внимание на то, что так как в данном случае у нас нет item-признаков, то параметр item_features задавать не нужно.

Вычислите показатель точности (precision@k) при k = 10. Ответ округлите до двух знаков после точки-разделителя.

In [132]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k  
from scipy.sparse import csr_matrix

In [138]:
train, test = random_train_test_split(
    ratings_matrix, 
    test_percentage=0.3, 
    random_state=13
    )
model = LightFM(no_components=100, 
                random_state=13, 
                learning_rate=0.05, 
                loss='warp')
model.fit(train)

precision_at_k(model, test).mean()

0.037982196

In [144]:
from numpy import dot
from numpy.linalg import norm

p = np.array([0.03, 4.41, 2.05])

a = np.array([3.29, 3.44, 3.67])
b = np.array([0.82, 9.71, 3.88])
c = np.array([8.34, 1.72, 0.02])

def cos_sim(a, b):
    return np.dot(a, b)/(norm(a) * norm(b))

print(cos_sim(p, a))
print(cos_sim(p, b))
print(cos_sim(p, c))


0.7797420921104519
0.9958958991578115
0.1901893919305701


In [146]:
(1 + 1/4 + 1/3)/3

0.5277777777777778